## Lesson 6 - Pretrained Model
**미션 개요**
- 이번 실습 자료에서는 강의시간에 다루었던 torchvision 을 사용하여 pretrained 모델을 사용하는 방법에 대해 실습하겠습니다.

**미션의 목적 및 배경**
- 파이토치에는 유명하고 근본이 되는 많은 pretrained 모델들을 쉽게 사용하기 위한 기능들이 많이 있습니다.
- 나만의 모델 아키텍쳐를 디자인하여 모델링을 시도하는 것도 좋은 방법이나, 이미 유명하고 많은 사람들이 사용하여 검증된 모델 아키텍쳐로 심지어 어느정도 학습도 된 pretrained 모델을 사용하여 모델링을 시작하는 것이 현명한 방법일 수 있습니다.


**미션 수행으로 얻어갈 수 있는 역량**
- 다양한 근본이 되는 pretrained 모델의 종류에 대해 배우고 이것들을 다운로드 받아서 load 하고 사용하는 방법들에 대해 배웁니다.
- 다른 유명 모델들은 layer 를 어떤 방식으로 쌓아가는지 살펴보면서 어떤 테크닉들과 노하우들이 있는지 배울 수 있습니다.

**미션 핵심 내용**
- torchvision 을 통해 다양한 pretrained 모델을 load 해보고 그 내부 구현을 살펴본다.


- torchvision 의 pretrained model 리스트는 다음과 같습니다
 
 [List of torchvision models](https://github.com/pytorch/vision/blob/main/torchvision/models/__init__.py#L1-L18)
```
from .alexnet import *
from .convnext import *
from .densenet import *
from .efficientnet import *
from .googlenet import *
from .inception import *
from .mnasnet import *
from .mobilenet import *
from .regnet import *
from .resnet import *
from .shufflenetv2 import *
from .squeezenet import *
from .vgg import *
from .vision_transformer import *
from .swin_transformer import *
from .maxvit import *
from . import detection, optical_flow, quantization, segmentation, video
from ._api import get_model, get_model_builder, get_model_weights, get_weight, list_models
```

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

#### 가장 기본이라고 할 수 있는 Alextnet 모델 아키텍쳐를 사용해보겠습니다.

In [ ]:
from torchvision.models import alexnet
model = alexnet()
model

#### Alexnet 의 pretrained 버전 또한 쉽게 불러올 수 있습니다.

In [ ]:
model = alexnet(pretrained=True)
model

#### torchvision 에서 해당 모델을 어떤 식으로 구현하였는지 직접 확인해보면 매우 도움이 많으 됩니다.
Example:
[source code](https://github.com/pytorch/vision/blob/master/torchvision/models/alexnet.py#L15-L50)
```
class AlexNet(nn.Module):

    def __init__(self, num_classes=1000):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x
```

#### 다른 모델들( e.g. vgg19, resnet18) 도 같은 방법으로 손 쉽게 사용할 수 있습니다.

In [ ]:
from torchvision.models import vgg19_bn
model = vgg19_bn(pretrained=True)
model

#### Pretrained 모델을 내 태스크에 맞게 어떻게 사용할 수 있나요?
 - Trochvision 모델들은 보통 feature-extraction 파트, task-specific 파트로 크게 두 가지로 구성되어 있습니다.
 - Task specific 파트는 모델의 태스크(이미지 분류, 객체 인식 등) 에 따라 모두 다릅니다.
 - 심지어 같은 이미지 분류 안에서도, 어떤 데이터셋으로 pretrain 하였느냐에 따라 다를 수 있습니다.
 - 따라서, 우리도 우리 테스크에 맞게 task specific 파트는 새로 정의하여 사용하여야 합니다.

 - 주로 이미지넷 데이터셋을 사용하여 pretrain 을 하기에 output_dim=1000 인 경우가 많습니다.
 - 따라서 우리 태스크의 클래스 갯수(18)에 맞게 재정의하여 사용할 수 있습니다.

In [ ]:
num_classes = 18
model = vgg19_bn(pretrained=True)
model.classifier = nn.Sequential(
    nn.Linear(512 * 7 * 7, 4096),
    nn.ReLU(True),
    nn.Dropout(),
    nn.Linear(4096, 4096),
    nn.ReLU(True),
    nn.Dropout(),
    nn.Linear(4096, num_classes),
)

model

#### Weight Freeze
 - Weight freeze 란 해당 모듈의 graident 는 역전파 하지 않아 학습을 하지 않는다는 의미입니다.
 - 예를 들어, 우리가 하려는 태스크가 pretrain 한 태스크와 매우 유사하다면, feature 파트는 freeze 하여 학습하지 않고 새로 정의한 task specific 파트만 학습하는 것이 좋은 방법일 수 있습니다.
 - weight freeze 는 `requires_grad` 를 사용하여 쉽게 구현할 수 있습니다. 

In [ ]:
# feature 파트만 freeze
model.features.requires_grad_(False)
for param, weight in model.named_parameters():
    print(f"파라미터 {param:20} 가 gradient 를 tracking 하나요? -> {weight.requires_grad}")

#### Weight initialization 
 - weight 초기화는 종종 모델의 성능에 critical 한 영향을 줍니다.
 - 하지만 만약 pretrained 모델을 사용한다면 pretrained 부분은 초기화를 하지 말고, 재정의한 task specific 파트만 초기화하여야 합니다.

In [ ]:
import torch.nn.init as init

def initialize_weights(model):
    """
    Xavier uniform 분포로 모든 weight 를 초기화합니다.
    더 많은 weight 초기화 방법은 다음 문서에서 참고해주세요. https://pytorch.org/docs/stable/nn.init.html
    """
    for m in model.modules():
        if isinstance(m, nn.Conv2d):
            init.xavier_uniform_(m.weight.data)
            if m.bias is not None:
                m.bias.data.zero_()
        elif isinstance(m, nn.BatchNorm2d):
            m.weight.data.fill_(1)
            m.bias.data.zero_()
        elif isinstance(m, nn.Linear):
            m.weight.data.normal_(0, 0.01)
            m.bias.data.zero_()

#### pretrained 모델을 가져와 가장 앞단 layer 의 weight 분포를 봐봅시다

In [ ]:
import matplotlib.pyplot as plt

model = vgg19_bn(pretrained=True)

# Weight Initialization 이전 모델 feature 파트의 첫번째 weight 분포
plt.hist(model.features[0].weight.detach().numpy().reshape(-1))  
plt.show()

#### weight 초기화 후 분포를 봐 봅시다
 - `xavier_uniform` 으로 초기화하여 웨이트들이 uniform 한 분포를 가지게 되었습니다.

In [ ]:
model = vgg19_bn(pretrained=True)

# 모든 weight 를 initialize
initialize_weights(model.features)

# Weight Initialization 이후 모델 feature 파트의 첫번째 weight 분포
# (xavier) uniform 한 분포로 바뀐 것을 확인할 수 있습니다.
plt.hist(model.features[0].weight.detach().numpy().reshape(-1))
plt.show()

#### task specific 한 부분만 초기화하엿습니다
 - feature extraction 파트는 초기화가 되지 않은 것은 확인할 수 있습니다.

In [ ]:
model = vgg19_bn(pretrained=True)

# Classifier 부분만 initialize
initialize_weights(model.classifier)

# Weight Initialization 이후 모델 feature 파트의 첫번째 weight 분포
# classifier 부분만 xavier uniform 으로 초기화해서 feature 파트는 uniform 한 분포를 가지지 않는 것을 확인할 수 있습니다.
plt.hist(model.features[0].weight.detach().numpy().reshape(-1)) 
plt.show()

## Appendix (optional)

### SOTA (State Of The Art)  모델을 리서치 하는 방법
- timm
- paper with code

## timm (pyTorch IMage Models)

PyTorch Image Models (timm) is a collection of image models, layers, utilities, optimizers, schedulers, data-loaders / augmentations, and reference training / validation scripts that aim to pull together a wide variety of SOTA models with ability to reproduce ImageNet training results.

#### References
https://github.com/rwightman/pytorch-image-models#introduction

https://fastai.github.io/timmdocs/

https://rwightman.github.io/pytorch-image-models/

In [ ]:
!pip install timm

#### Timm 을 사용하여 pretrained 모델 불러오기

In [ ]:
import timm

m = timm.create_model('mobilenetv3_large_100', pretrained=True)
m.eval()

#### Timm 에서 사용가능한 pretrained 모델 목록

In [ ]:
import timm
from pprint import pprint
model_names = timm.list_models(pretrained=True)
pprint(model_names)

#### 다음과 같은 방법을 통해서 원하는 모델을 찾는 것도 가능합니다

In [ ]:
import timm
from pprint import pprint
model_names = timm.list_models('*resne*t*')
pprint(model_names)

## Paper with code
 - https://paperswithcode.com/task/image-classification
 - 다양한 태스크와 데이터셋에 대한 다양한 모델들의 성능을 벤치마킹해주는 웹서비스입니다.
 - 해당 서비스를 통해 각 모델들의 성능 비교뿐 아니라 논문과 구현 코드로 forwarding 도 가능합니다.

## 레이어 직접 쌓아보기

![python image2](https://cphinf.pstatic.net/mooc/20210813_264/1628827925318KzHFu_JPEG/mceclip0.jpg)

해당 모델 아키텍쳐는 향후 배우시게 될 Object Detection 쪽에서 하나의 큰 계보라 할 수 있는 yolo 의 backbone 으로 많이 사용하는 Darknet53 입니다

Darknet53 은 ResidualBlock 을 해상도를 줄여가며(receptive field 를 늘려가며) 쌓은 구조를 가지고 있습니다.

마지막 FC 레이어를 제외하고는 Feature Extraction Layer 로도 다양하게 활용할 수 있습니다.

In [ ]:
import torch
from torch import nn

def conv_batch(in_num, out_num, kernel_size=3, padding=1, stride=1):
    return nn.Sequential(
        nn.Conv2d(in_num, out_num, kernel_size=kernel_size, stride=stride, padding=padding, bias=False),
        nn.BatchNorm2d(out_num),
        nn.LeakyReLU())


# Residual block
class DarkResidualBlock(nn.Module):
    def __init__(self, in_channels):
        super(DarkResidualBlock, self).__init__()

        reduced_channels = int(in_channels/2)

        self.layer1 = conv_batch(in_channels, reduced_channels, kernel_size=1, padding=0)
        self.layer2 = conv_batch(reduced_channels, in_channels)

    def forward(self, x):
        residual = x

        out = self.layer1(x)
        out = self.layer2(out)
        out += residual
        return out


class Darknet53(nn.Module):
    def __init__(self, block, num_classes):
        super(Darknet53, self).__init__()

        self.num_classes = num_classes

        self.features = nn.Sequential(
            conv_batch(3, 32),
            conv_batch(32, 64, stride=2),
            self.make_layer(block, in_channels=64, num_blocks=1),
            conv_batch(64, 128, stride=2),
            self.make_layer(block, in_channels=128, num_blocks=2),
            conv_batch(128, 256, stride=2),
            self.make_layer(block, in_channels=256, num_blocks=8),
            conv_batch(256, 512, stride=2),
            self.make_layer(block, in_channels=512, num_blocks=8),
            conv_batch(512, 1024, stride=2),
            self.make_layer(block, in_channels=1024, num_blocks=4),
        )
        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Linear(1024, self.num_classes)

    def forward(self, x):
        out = self.features(x)
        out = self.global_avg_pool(out)
        out = out.view(-1, 1024)
        out = self.fc(out)

        return out

    def make_layer(self, block, in_channels, num_blocks):
        layers = []
        for i in range(0, num_blocks):
            layers.append(block(in_channels))
        return nn.Sequential(*layers)


def darknet53(num_classes):
    return Darknet53(DarkResidualBlock, num_classes)

In [ ]:
net = darknet53(num_classes=18)
net

Darknet53(
  (features): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
    (1): Sequential(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
    (2): Sequential(
      (0): DarkResidualBlock(
        (layer1): Sequential(
          (0): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): LeakyReLU(negative_slope=0.01)
        )
        (layer2): Sequential(
          (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (1): BatchNorm2d(64, eps

###**콘텐츠 라이선스**

<font color='red'><b>**WARNING**</b></font> : **본 교육 콘텐츠의 지식재산권은 재단법인 네이버커넥트에 귀속됩니다. 본 콘텐츠를 어떠한 경로로든 외부로 유출 및 수정하는 행위를 엄격히 금합니다.** 다만, 비영리적 교육 및 연구활동에 한정되어 사용할 수 있으나 재단의 허락을 받아야 합니다. 이를 위반하는 경우, 관련 법률에 따라 책임을 질 수 있습니다.